<a href="https://colab.research.google.com/github/BaishaliChetia/CapsNet-Keras/blob/master/CompressionComparisons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project subjects the Capsule network and a VGG16 (convolutional nueral network) to model optimiztion techniques such a pruning and quantization and comparest them with respect to Latency , Accuracy and Model Size. 

# Get Proessor Info

In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
stepping	: 3
microcode	: 0x1
cpu MHz		: 2000.128
cache size	: 39424 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm mpx avx512f avx512dq rdseed adx smap clflushopt clwb avx512cd avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	:

In [ ]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         148G   39G  110G  26% /
tmpfs            64M     0   64M   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
shm             5.8G     0  5.8G   0% /dev/shm
/dev/sda1       154G   41G  114G  27% /opt/bin/.nvidia
tmpfs           6.4G   36K  6.4G   1% /var/colab
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware


In [ ]:
!cat /proc/meminfo

MemTotal:       13302924 kB
MemFree:        10359412 kB
MemAvailable:   12456960 kB
Buffers:          146316 kB
Cached:          2091884 kB
SwapCached:            0 kB
Active:          1039664 kB
Inactive:        1608108 kB
Active(anon):     379420 kB
Inactive(anon):      484 kB
Active(file):     660244 kB
Inactive(file):  1607624 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:             18692 kB
Writeback:             0 kB
AnonPages:        409588 kB
Mapped:           237836 kB
Shmem:              1180 kB
KReclaimable:     142312 kB
Slab:             193924 kB
SReclaimable:     142312 kB
SUnreclaim:        51612 kB
KernelStack:        4992 kB
PageTables:         5552 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6651460 kB
Committed_AS:    3027928 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       47184 kB
VmallocChunk:          0 kB
Percpu:          

In [ ]:
!nvidia-smi
print()
print()
print('=' * 80)
print()
print()
!df -h
print()
print()
print('=' * 80)
print()
print()
!free -m
print()
print()
print('=' * 80)
print()
print()
!lscpu
print()
print()
print('=' * 80)
print()
print()
! ps -eo pmem,pcpu,vsize,pid,cmd | sort -k 1 -nr | head -5

Wed Aug 25 09:42:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import tensorflow.keras as K
#import tensorflow_model_optimization as tfmot

In [ ]:
pip install -q tensorflow-model-optimization

     |████████████████████████████████| 211 kB 5.1 MB/s 


In [ ]:
import tensorflow_model_optimization as tfmot

# Capsule Network for 28X28 image size
first we build a capsule network and train it on mnist dataset with image size, and subject it to compression and check its performance with respect to accuracy, latency and model size. 
Please check the original code on Capsule network for detailed description of each function 
https://github.com/BaishaliChetia/MyProjects/blob/2452391b1dc6699979d1928c75d01fcf24aa6f9b/DeepLearning/CapsuleNetwork_pruned_quantized.ipynb

In [ ]:
caps1_n_maps = 32
caps1_n_caps = caps1_n_maps * 6 * 6  # 1152 primary capsules , for 28X28 image size
caps1_n_dims = 8
caps2_n_caps = 10
caps2_n_dims = 16

tf.random.set_seed(500000)

In [ ]:
#class SquashLayer(K.layers.Layer, tfmot.sparsity.keras.PrunableLayer):
class SquashLayer(K.layers.Layer):
  def __init__(self, axis=-1, **kwargs):
    super(SquashLayer, self).__init__(**kwargs)
    self.axis = axis
    
  def build(self, input_shapes):
    pass

  """
  def get_prunable_weights(self):
    return []
  """

  def call(self, inputs):
    EPSILON = 1.0e-9
    squared_norm = tf.compat.v1.reduce_sum(tf.square(inputs),\
                                           axis=self.axis,\
                                           keepdims=True)
    safe_norm = tf.sqrt(squared_norm + EPSILON)
    squash_factor = squared_norm / (1. + squared_norm)
    unit_vector = inputs / safe_norm
    return squash_factor * unit_vector

  def get_config(self):
    config = super(SquashLayer, self).get_config()
    config.update({"axis": self.axis})
    return config

 
#class SafeNorm(K.layers.Layer, tfmot.sparsity.keras.PrunableLayer):
class SafeNorm(K.layers.Layer):
  
  def __init__(self, axis=-1, keep_dims = False,  **kwargs):
    super(SafeNorm, self).__init__(**kwargs)
    self.axis = axis
    self.keep_dims = keep_dims

  def build(self, input_shapes):
    pass

  """
  def get_prunable_weights(self):
    return []
  """

  def call(self, input):
    EPSILON = 1.0e-9
    squared_norm = tf.compat.v1.reduce_sum(tf.square(inputs),\
                                           axis=self.axis,\
                                           keepdims= self.keep_dims)
    safe_norm = tf.sqrt(squared_norm + EPSILON)
    return safe_norm

  def get_config(self):
    config = super(SafeNorm, self).get_config()
    config.update({"axis": self.axis, "keep_dims": self.keep_dims})
    return config
  
# This should be the part where the digit layer, and where we tile things
# This is incomplete, and work in progress
# TODO: Complete this
class MyDigitCapsLayer(K.layers.Layer, tfmot.sparsity.keras.PrunableLayer):
  def __init__(self, **kwargs):
    super(MyDigitCapsLayer, self).__init__(**kwargs)
  
  def get_config(self):
    config =  super(MyDigitCapsLayer, self).get_config()
    return config
  

  def build(self, input_shapes):
    init_sigma = 0.1  # TODO: use
    self.kernel = self.add_weight(\
                      "kernel",\
                      (caps1_n_caps, caps2_n_caps, caps2_n_dims, caps1_n_dims),\
                      initializer="random_normal",\
                      dtype=tf.float32)

  
  # To debug this function, I used prints to print the shape
  # expand_dims just adds an exis, so if you say expand_dims(inshape=(5, 3), -1),
  # you get the output shape (5, 3, 1), it just adds an axis at the end
  # Then tile just multiplies one of the dimensions (that is it stacks along that direction N times)
  # so tile(inshape=(5, 3, 1), [1, 1, 1000]) will yield a shape (5, 3, 1000)
  #
  # Notice I didn't tile in build, but in call, Most probaly this is the right thing to do
  # but we'll only figure out when we actually train
  def get_prunable_weights(self):
    return [self.kernel]
  
  def call(self, inputs):
    # Add a dimension at the end
    exp1 = tf.expand_dims(inputs, -1, name="caps1_output_expanded")
    # add a dimension along 3rd axis
    exp1 = tf.expand_dims(exp1, 2, name="caps2_output_espanced")
    # tile along 3rd axis
    tile = tf.tile(exp1, [1, 1, caps2_n_caps, 1, 1], name="caps1_output_tiled")
    caps2_predicted = tf.matmul(self.kernel, tile, name="caps2_predicted")
    return caps2_predicted

# https://www.tensorflow.org/api_docs/python/tf/keras/losses/Loss
class MarginLoss(K.losses.Loss):
    def __init__(self, **kwargs):
      super(MarginLoss, self).__init__(**kwargs)

  
    def get_config(self):
      config = super(MarginLoss, self).get_config()
      return config
    
    def safe_norm(self, input, axis=-2, epsilon=1e-5, keep_dims=False, name=None):
      squared_norm = tf.reduce_sum(tf.square(input), axis=axis,
                                     keepdims=keep_dims)
      return tf.sqrt(squared_norm + epsilon)

    """
    def get_prunable_weights(self):
      return []
    """

    def call(self,y_true, input):
      # print(f"y_true.shape = {y_true.shape}, y_pred.shape = {y_pred.shape}")
      # return K.losses.MeanSquaredError()(y_true, y_pred)

      #y_true = K.Input(shape=[], dtype=tf.int64, name="y")
      m_plus = 0.9
      m_minus = 0.1
      lambda_ = 0.5 
      
      #y_true one hot encode y_train
      T = tf.one_hot(y_true, depth=caps2_n_caps, name="T")
      
      caps2_output_norm = self.safe_norm(input, keep_dims = True)

      present_error_raw = tf.square(\
                                    tf.maximum(0., m_plus - caps2_output_norm),
                                    name="present_error_raw")
      present_error = tf.reshape(\
                                    present_error_raw, shape=(-1, 10),
                                    name="present_error")  
  
      absent_error_raw = tf.square(\
                                    tf.maximum(0., caps2_output_norm - m_minus),
                                    name="absent_error_raw")
      absent_error = tf.reshape(\
                                    absent_error_raw, shape=(-1, 10),
                                    name="absent_error")
  
      L = tf.add(\
                  T * present_error,\
                  lambda_ * (1.0 - T) * absent_error,
                  name="L")
      marginLoss = tf.reduce_mean(\
                                  tf.reduce_sum(L, axis=1),\
                                  name="margin_loss")
      return marginLoss


#class RoutingByAgreement(K.layers.Layer, tfmot.sparsity.keras.PrunableLayer):
class RoutingByAgreement(K.layers.Layer):
  def __init__(self, round_number=-1, **kwargs):
    super(RoutingByAgreement, self).__init__(**kwargs)
    self.round_number = round_number 

  def get_config(self):
    config = super(RoutingByAgreement, self).get_config()
    config.update({"round_number": self.round_number})
    return config
 

  def build(self, input_shapes):
    self.raw_weights_1 = self.add_weight("raw_weights", \
                                         (caps1_n_caps, caps2_n_caps, 1, 1), \
                                         initializer = "zeros", \
                                         dtype=tf.float32,)
    
    #print("Routing layer: self.raw_weights = ", self.raw_weights.shape, "input_shape = ", input_shapes)

  
  def get_prunable_weights(self):
    return [self.raw_weights_1]


  @staticmethod
  def squash(inputs):
    EPSILON = 1.0e-5
    squared_norm = tf.compat.v1.reduce_sum(tf.square(inputs),\
                                           keepdims=True)
    safe_norm = tf.sqrt(squared_norm + EPSILON)
    squash_factor = squared_norm / (1. + squared_norm)
    unit_vector = inputs / safe_norm
    return squash_factor * unit_vector

  def single_round_routing(self, inputs, raw_weights, agreement):
    raw_weights = tf.add(raw_weights, agreement)
    routing_wt = tf.nn.softmax(raw_weights, axis=2)
    wt_predictions = tf.multiply(routing_wt, inputs)
    wt_sum = tf.reduce_sum(wt_predictions, axis=1, keepdims=True)
    return wt_sum

  def call(self, inputs):
    agreement = tf.zeros(shape=self.raw_weights_1.shape)
    sqsh_wt_sum = None
    x = inputs
    for i in range(2):
      wt_sum = self.single_round_routing(inputs, self.raw_weights_1, agreement)
      sqsh_wt_sum = RoutingByAgreement.squash(wt_sum)
      sqsh_wt_sum_tiled = tf.tile(\
                          sqsh_wt_sum ,\
                          [1, caps1_n_caps, 1, 1, 1],\
                          name="caps2_output_round_1_tiled")
      agreement = tf.matmul(\
                            x, \
                            sqsh_wt_sum_tiled,\
                            transpose_a=True,\
                            name="agreement")
    return sqsh_wt_sum

class MyAccuracy(K.metrics.Metric):
  def __init__(self, **kwargs):
    super(MyAccuracy, self).__init__(**kwargs)
    self.acc_obj = K.metrics.Accuracy()
    self.state = 0
  

  def get_config(self):
    config = super(MyAccuracy, self).get_config()
    config.update({"acc_obj": None, "state": self.state})
    return config
  

  def safe_norm(self, input, axis=-2, epsilon=1e-5, keep_dims=True, name=None):
      squared_norm = tf.reduce_sum(tf.square(input), axis=axis,
                                     keepdims=keep_dims)
      return tf.sqrt(squared_norm + epsilon)

  def update_state(self, y_true, input, sample_weight=None):
    if self.acc_obj is None:
      self.acc_obj = K.metrics.Accuracy()
    y_proba = self.safe_norm(input, axis=-2)
    y_proba_argmax = tf.argmax(y_proba, axis=2)
    y_pred = tf.squeeze(y_proba_argmax, axis=[1,2], name="y_pred")
    #y_true = tf.reshape(y_true, (y_true.shape[0], ))
    y_true = tf.cast(y_true, dtype=tf.int64)
    self.acc_obj.update_state(y_true, y_pred, sample_weight)

  def reset_state(self):
    self.acc_obj.reset_state()

  def result(self):
    return self.acc_obj.result()

class MyReshapeLayer(K.layers.Layer):
  def __init__(self, axis=-1, keep_dims = False,  **kwargs):
    super(MyReshapeLayer, self).__init__(**kwargs)

  def build(self, input_shapes):
    pass

  def safe_norm(self, input, axis=-2, epsilon=1e-5, keep_dims=True, name=None):
      squared_norm = tf.reduce_sum(tf.square(input), axis=axis,
                                     keepdims=keep_dims)
      return tf.sqrt(squared_norm + epsilon)

  def call(self, input):
    print('printing shapes ------------------- ')
    EPSILON = 1.0e-9
    print(input)
    y_proba = self.safe_norm(input, axis=-2)
    print(y_proba)
    y_proba_argmax = tf.argmax(y_proba, axis=2)
    print(y_proba_argmax)
    y_pred = tf.squeeze(y_proba_argmax, axis=[1,2], name="y_pred")
    print(y_pred)
    return tf.cast(y_pred, tf.int64)

  def get_config(self):
    config = super(MyReshapeLayer, self).get_config()
    return config
  

In [ ]:
(x_train, y_train,), (x_test, y_test) = K.datasets.mnist.load_data()
#print(x_train.shape, x_test.shape)
x_train = x_train/255.0
x_test = x_test/255.0

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
class Model:
    @staticmethod
    def build(inshape=(28, 28, 1)):
        inp = K.Input(shape=inshape, dtype=tf.float32, name='input')
        
        # Primary capsules
        # For each digit in the batch
        # 32 maps, each 6x6 grid of 8 dimensional vectors
        
        # First Conv layer
        conv1_params = \
        {
            "filters": 256,
            "kernel_size": 9,
            "strides": 1,
            "padding": "valid",
            "activation": tf.nn.relu,
        }
        x = K.layers.Conv2D(**conv1_params, name="conv_layer_1")(inp)
        
        # Second conv layer
        conv2_params = \
        {
            "filters": caps1_n_maps * caps1_n_dims, # 256 convolutional filters
            "kernel_size": 9,
            "strides": 2,
            "padding": "valid",
            "activation": tf.nn.relu
        }
        x = K.layers.Conv2D(**conv2_params, name="conv_layer_2")(x)
        
        # Reshape
        x = K.layers.Reshape(\
                             (caps1_n_caps, caps1_n_dims),\
                             name="reshape_layer_1")(x)
                             
        x = SquashLayer(name="caps1_output_layer")(x)
        
        x = MyDigitCapsLayer(name = "caps2_predicted")(x)
        caps2_predicted = x # Save this value for later
        
        #routing by agreement (2 rounds)
        x = RoutingByAgreement(name="routing1", round_number=2)(x)
        
        return K.Model(inputs=inp, outputs=x, name='my')
    
m = Model.build()
print(m.summary())


Model: "my"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv_layer_1 (Conv2D)        (None, 20, 20, 256)       20992     
_________________________________________________________________
conv_layer_2 (Conv2D)        (None, 6, 6, 256)         5308672   
_________________________________________________________________
reshape_layer_1 (Reshape)    (None, 1152, 8)           0         
_________________________________________________________________
caps1_output_layer (SquashLa (None, 1152, 8)           0         
_________________________________________________________________
caps2_predicted (MyDigitCaps (None, 1152, 10, 16, 1)   1474560   
_________________________________________________________________
routing1 (RoutingByAgreement (None, 1, 10, 16, 1)      11520    

In [ ]:
m.compile(optimizer='adam', loss=MarginLoss(), metrics=[MyAccuracy()])

In [ ]:
m.load_weights("/content/drive/MyDrive/MnistResults/best_weights4.hdf5")

In [ ]:
_, baseline_model_accuracy = m.evaluate(
    x_test, y_test, verbose=1)

313/313 [==============================] - 26s 33ms/step - loss: 2.2766 - my_accuracy: 0.9837


In [ ]:
y_pred = m.predict(x_test)

import sklearn
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
print(confusion_matrix(y_test, y_pred))
print(f"accuracy = {accuracy_score(y_test, y_pred)}")

NameError: ignored

In [ ]:
print("For 28X28 image, evaluation time per test instance: " + str((26/x_test.shape[0])*1000) + "ms")

For 28X28 image, evaluation time per test instance: 2.6ms


In [ ]:
# class TimeHistory(K.callbacks.Callback):
#     def on_train_begin(self, logs={}):
#         self.times = []

#     def on_epoch_begin(self, batch, logs={}):
#         self.epoch_time_start = time.time()

#     def on_epoch_end(self, batch, logs={}):
#         self.times.append(time.time() - self.epoch_time_start)
# time_callback = TimeHistory()
# history = m.fit(x_train, y_train, batch_size=32, epochs=50, verbose= 1, validation_split=0.2, callbacks = [time_callback])
# times = time_callback.times
# print(times)

In [ ]:
# t1 = time.perf_counter()
# for i in range(x_test.shape[0]):
#   y_pred[i] = newmodel.predict(x_test[i])
#   print(f"accuracy = {accuracy_score(y_test[i], y_pred[i])}")
# t2 = time.perf_counter()
# per_instance_time = (t2-t1)/x_test.shape[0]
# print(per_instance_time)


In [ ]:
#m.save("/content/drive/MyDrive/WeightsMnist/save.tf", save_format='tf')

# 
#!rm -f "/content/drive/MyDrive/WeightsMnist/save3.tf"

rm: cannot remove '/content/drive/MyDrive/WeightsMnist/save3.tf': Is a directory


In [ ]:
basemodel_file = m.save("/content/drive/MyDrive/MnistResults/caps_basemodelB.tf", save_format='tf')


INFO:tensorflow:Assets written to: /content/drive/MyDrive/MnistResults/caps_basemodelB.tf/assets


In [ ]:
# model = tf.keras.models.load_model("/content/drive/MyDrive/MnistResults/save_basemodel5.tf", 
#                                    custom_objects=\
#                           {\
#                               "SquashLayer": SquashLayer,\
#                               "SafeNorm": SafeNorm,\
#                               "MyDigitCapsLayer": MyDigitCapsLayer,
#                               "RoutingByAgreement": RoutingByAgreement,                      
#                               "MyAccuracy": MyAccuracy,                            
#                               "MarginLoss": MarginLoss
#                            })


mm = K.models.load_model('/content/drive/MyDrive/WeightsMnist/save2.tf',\
                         custom_objects=\
                          {\
                              "SquashLayer": SquashLayer,\
                              "SafeNorm": SafeNorm,\
                              "MyDigitCapsLayer": MyDigitCapsLayer,\
                              "RoutingByAgreement": RoutingByAgreement,\
                              "MyAccuracy": MyAccuracy,\
                              "MarginLoss": MarginLoss,\
                          })


In [ ]:
# mm = K.models.load_model('/content/drive/MyDrive/WeightsMnist/save2.tf',\
#                          custom_objects=\
#                           {\
#                               "SquashLayer": SquashLayer,\
#                               "SafeNorm": SafeNorm,\
#                               "MyDigitCapsLayer": MyDigitCapsLayer,\
#                               "RoutingByAgreement": RoutingByAgreement,\
#                               "MyAccuracy": MyAccuracy,\
#                               "MarginLoss": MarginLoss,\
#                           })


In [ ]:
# m3 = K.models.Sequential()
# m3.add(m2)
# m3.add(y_pred_eval)
# m3.build()
# m3.compile(optimizer='adam', loss=MarginLoss(), metrics=[MyAccuracy()])
# m3.evaluate(x_test, y_test, batch_size= 32, verbose= 1)

# #m3.evaluate(x_test, y_test, batch_size= 32, verbose= 1)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(m)

In [ ]:
mm = m

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
quantize_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp2v__ltcn/assets


INFO:tensorflow:Assets written to: /tmp/tmp2v__ltcn/assets


In [ ]:
# Create the .tflite file
tflite_model_file = "/content/drive/MyDrive/MnistResults/caps_compressedB.tflite"
converter = tf.lite.TFLiteConverter.from_keras_model(mm)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]

tflite_model = converter.convert()
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpf79v0cle/assets


INFO:tensorflow:Assets written to: /tmp/tmpf79v0cle/assets


# caps_basemodel.B = 27M
# caps_compressedB (no pruning)  = 11MB

In [ ]:
#checking for model 4
!du -sh /content/drive/MyDrive/MnistResults/*

27M	/content/drive/MyDrive/MnistResults/best_weights4.hdf5
11M	/content/drive/MyDrive/MnistResults/compressed1.tflite
27M	/content/drive/MyDrive/MnistResults/latest_weights1.hdf5
27M	/content/drive/MyDrive/MnistResults/latest_weights2.hdf5
27M	/content/drive/MyDrive/MnistResults/latest_weights3.hdf5
27M	/content/drive/MyDrive/MnistResults/latest_weights4.hdf5
512	/content/drive/MyDrive/MnistResults/mylogs1.csv
512	/content/drive/MyDrive/MnistResults/mylogs2.csv
512	/content/drive/MyDrive/MnistResults/mylogs3.csv
5.5K	/content/drive/MyDrive/MnistResults/mylogs4.csv
27M	/content/drive/MyDrive/MnistResults/pruned_file1.h5
27M	/content/drive/MyDrive/MnistResults/pruned_file2.h5
79M	/content/drive/MyDrive/MnistResults/save_basemode2.tf
79M	/content/drive/MyDrive/MnistResults/save_basemode3.tf
79M	/content/drive/MyDrive/MnistResults/save_basemodel4.tf
79M	/content/drive/MyDrive/MnistResults/save_basemodel.tf


# Use this tutorial for pruning
quantization has already been done earlier

https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras


# PRUNING


In [ ]:
# pip install -q tensorflow-model-optimization

In [ ]:
comparison_metric.name


'my_accuracy_8'

In [ ]:
import tensorflow_model_optimization as tfmot
import tempfile

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# TODO: Remove this line
#m.load_weights('/content/drive/MyDrive/MnistResults/best_weights4.hdf5')

# print("ORIGINAL MODEL")
# print(mm.summary())
# print('-' * 80)

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = x_train.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

mm = m
mm.load_weights('/content/drive/MyDrive/MnistResults/best_weights4.hdf5')

# Define model for pruning.
# pruning_params = {
#       'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
#                                                                final_sparsity=0.80,
#                                                                begin_step=0,
#                                                                end_step=end_step)
# }

pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.20,
                                                              final_sparsity=0.65,
                                                              begin_step=0,
                                                              end_step=end_step)
}

"""
model_for_pruning = prune_low_magnitude(mm, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=MarginLoss(), metrics=[MyAccuracy()])

model_for_pruning.summary()

"""


######################################################################
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

# Helper function uses `prune_low_magnitude` to make only the 
# Dense layers train with pruning.
def apply_pruning_to_layers(layer):
  #print("called")
  if isinstance(layer, MyDigitCapsLayer):
    print(f"Layer {layer} {layer.name} slated for pruning")
    return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
  elif layer.name == "conv_layer_2":
    print(f"Layer {layer} {layer.name} slated for pruning")
    return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
  elif layer.name == "routing1":
    print(f"Layer {layer} {layer.name} slated for pruning")
    return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
  elif layer.name == "conv_layer_1":
    print(f"Layer {layer} {layer.name} slated for pruning")
    return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)


  print(f"Layer {layer} {layer.name} unchanged")
  return layer

# Use `tf.keras.models.clone_model` to apply `apply_pruning_to_layers` 
# to the layers of the model.
model_for_pruning = tf.keras.models.clone_model(
    mm,
    clone_function=apply_pruning_to_layers,
)
#print(model_for_pruning.summary())

"""
model_for_pruning = K.models.Sequential(\
              [\
                model_for_pruning,\
                MyReshapeLayer(),\
              ]\
)
"""


model_for_pruning.compile(optimizer='adam', loss=MarginLoss(), metrics=[MyAccuracy()])
#model_for_pruning.compile(optimizer='adam', loss=MarginLoss())

model_for_pruning.fit(x_train, y_train,
                  batch_size=32, epochs=2, validation_split=validation_split,
                  callbacks=callbacks, verbose = 1)
_, model_for_pruning_accuracy = model_for_pruning.evaluate(x_test, y_test, verbose=1)


Layer <keras.layers.convolutional.Conv2D object at 0x7f2171de4a50> conv_layer_1 slated for pruning
Layer <keras.layers.convolutional.Conv2D object at 0x7f2171de4b50> conv_layer_2 slated for pruning
Layer <keras.layers.core.Reshape object at 0x7f2171de49d0> reshape_layer_1 unchanged
Layer <__main__.SquashLayer object at 0x7f21607677d0> caps1_output_layer unchanged
Layer <__main__.MyDigitCapsLayer object at 0x7f2171eb3190> caps2_predicted slated for pruning
Layer <__main__.RoutingByAgreement object at 0x7f21606d6c10> routing1 slated for pruning


/usr/local/lib/python3.7/dist-packages/keras/engine/base_layer.py:2223: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Epoch 1/2
   6/1688 [..............................] - ETA: 5:56 - loss: 2.2786 - my_accuracy_2: 1.0000WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0686s vs `on_train_batch_end` time: 0.0773s). Check your callbacks.


1688/1688 [==============================] - 144s 84ms/step - loss: 2.2778 - my_accuracy_2: 0.9484 - val_loss: 2.2755 - val_my_accuracy_2: 0.9708
Epoch 2/2
313/313 [==============================] - 11s 34ms/step - loss: 2.2765 - my_accuracy_2: 0.9763


In [ ]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(x_test, y_test, verbose=1)

313/313 [==============================] - 10s 33ms/step - loss: 2.2765 - my_accuracy_2: 0.9763


In [ ]:
print("prunned model evaluation time:" + str((10/x_test.shape[0])*1000) + "ms, for 28X28 image" )

prunned model evaluation time:1.0ms, for 28X28 image


## prunned model evaluation time:1.0ms, for 28X28 image


#explanation for increase in parameters after pruning

https://www.dlology.com/blog/how-to-compress-your-keras-model-x5-smaller-with-tensorflow-model-optimization/

In [ ]:
# _, model_for_pruning_accuracy = model_for_pruning.evaluate(
#    x_test, y_test, verbose=1)

313/313 [==============================] - 11s 34ms/step - loss: 2.2769 - my_accuracy_72: 0.5371


In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
model_for_export.summary()
pruned_keras_file = "/content/drive/MyDrive/MnistResults/caps_pruned_fileB.h5"
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Model: "my"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv_layer_1 (Conv2D)        (None, 20, 20, 256)       20992     
_________________________________________________________________
conv_layer_2 (Conv2D)        (None, 6, 6, 256)         5308672   
_________________________________________________________________
reshape_layer_1 (Reshape)    (None, 1152, 8)           0         
_________________________________________________________________
caps1_output_layer (SquashLa (None, 1152, 8)           0         
_________________________________________________________________
caps2_predicted (MyDigitCaps (None, 1152, 10, 16, 1)   1474560   
_________________________________________________________________
routing1 (RoutingByAgreement (None, 1, 10, 16, 1)      11520    

Saved pruned Keras model to: /content/drive/MyDrive/MnistResults/caps_pruned_fileB.h5


# size of pruned model = 27M

In [ ]:
from tensorflow.keras.models import load_model

model = model_for_export
import numpy as np

for i, w in enumerate(model.get_weights()):
  print("{} -- Total:{}, Zeros: {:.2f}%".format(model.weights[i].name, w.size, np.sum(w == 0) / w.size * 100))
    

conv_layer_1/kernel:0 -- Total:20736, Zeros: 64.99%
conv_layer_1/bias:0 -- Total:256, Zeros: 0.00%
conv_layer_2/kernel:0 -- Total:5308416, Zeros: 64.99%
conv_layer_2/bias:0 -- Total:256, Zeros: 0.00%
caps2_predicted/kernel:0 -- Total:1474560, Zeros: 64.99%
routing1/raw_weights:0 -- Total:11520, Zeros: 64.99%


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]

tflite_model = converter.convert()


INFO:tensorflow:Assets written to: /tmp/tmp4ppdoc68/assets


INFO:tensorflow:Assets written to: /tmp/tmp4ppdoc68/assets


AttributeError: ignored

tyoes of post training quantization
https://www.tensorflow.org/lite/performance/post_training_quantization

In [ ]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [ ]:
#converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
tflite_model_file = "/content/drive/MyDrive/MnistResults/caps_P&Q_1.tflite"
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]

tflite_model = converter.convert()
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)



INFO:tensorflow:Assets written to: /tmp/tmpf4v5i690/assets


INFO:tensorflow:Assets written to: /tmp/tmpf4v5i690/assets


In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size("/content/drive/MyDrive/MnistResults/best_weights4.hdf5")))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(tflite_model_file)))

Size of gzipped baseline Keras model: 25639770.00 bytes
Size of gzipped pruned Keras model: 11977511.00 bytes
Size of gzipped pruned TFlite model: 4750440.00 bytes


In [ ]:
def get_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  #import zipfile

  _, model_file = tempfile.mkstemp('.tflite')
  with(model_file, 'w') as f:
    f.write(file)

  return os.path.getsize(file)

In [ ]:
tflite_model_file = "/content/drive/MyDrive/MnistResults/caps_P&Q_1_noZip.tflite"
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]

tflite_model = converter.convert()
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpxygeqd7y/assets


INFO:tensorflow:Assets written to: /tmp/tmpxygeqd7y/assets


In [ ]:
!du -sh /content/drive/MyDrive/MnistResults/*

27M	/content/drive/MyDrive/MnistResults/best_weights4.hdf5
27M	/content/drive/MyDrive/MnistResults/best_weights5.hdf5
31M	/content/drive/MyDrive/MnistResults/caps32X32_best_weights5.hdf5
3.0K	/content/drive/MyDrive/MnistResults/caps48X48mylogs5.csv
27M	/content/drive/MyDrive/MnistResults/caps_basemodel28X28_1.h5
1.0K	/content/drive/MyDrive/MnistResults/caps_basemodelA.h5
27M	/content/drive/MyDrive/MnistResults/caps_basemodelA.tf
27M	/content/drive/MyDrive/MnistResults/caps_basemodelB.tf
27M	/content/drive/MyDrive/MnistResults/caps_best_weightsA.hdf5
11M	/content/drive/MyDrive/MnistResults/caps_compressedA.tflite
11M	/content/drive/MyDrive/MnistResults/caps_compressedB.tflite
27M	/content/drive/MyDrive/MnistResults/caps_latest_weightsA.hdf5
4.0K	/content/drive/MyDrive/MnistResults/caps_mylogsA.csv
11M	/content/drive/MyDrive/MnistResults/caps_P&Q_1_noZip.tflite
11M	/content/drive/MyDrive/MnistResults/caps_P&Q_1.tflite
27M	/content/drive/MyDrive/MnistResults/caps_pruned_fileA.h5
27M	/cont

In [ ]:
keras_file = "/content/drive/MyDrive/MnistResults/caps_basemodel28X28_1.h5"
tf.keras.models.save_model(m, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Saved baseline model to: /content/drive/MyDrive/MnistResults/caps_basemodel28X28_1.h5


In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(tflite_model_file)))

Size of gzipped baseline Keras model: 11977519.00 bytes
Size of gzipped pruned Keras model: 11977511.00 bytes
Size of gzipped pruned TFlite model: 4750452.00 bytes


## ==================================================================================================================================

# VGG compression 32X32

In [ ]:
# Load MNIST dataset
import keras
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0
train_X =[]

for i in range(train_images.shape[0]):
  train_pad = np.pad(train_images[i], pad_width=2 , mode= 'edge')
  #print(train_pad)
  train_X.append(train_pad)
#PADDING test images
test_X = []
for i in range(test_images.shape[0]):
  test_pad = np.pad(test_images[i], pad_width=2 , mode= 'edge')
  #print(train_pad)
  test_X.append(test_pad)
train_X = np.array(train_X)
test_X = np.array(test_X)


11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
train_X=np.dstack([train_X] * 3)
test_X=np.dstack([test_X]*3)
train_X.shape,test_X.shape

((60000, 32, 96), (10000, 32, 96))

In [ ]:
train_X = train_X.reshape(-1, 32,32,3)
test_X= test_X.reshape (-1,32,32,3)

train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X.shape,test_X.shape

((60000, 32, 32, 3), (10000, 32, 32, 3))

In [ ]:
vggModel = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
vggModel.trainable = True
model2 = keras.Sequential([
  vggModel,
  keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.5),
  keras.layers.Dense(256, activation='relu'),
  keras.layers.Dense(50, activation = 'relu'),
  keras.layers.Dense(10, activation = 'softmax')
])
# Train the digit classification model
model2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# history = model2.fit(
#   train_X,
#   train_labels,
#   epochs= 50,
#   validation_split=0.1, callbacks = callback_list
# )


58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dense_1 (Dense)              (None, 50)                12850     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                510       
Total params: 14,859,376
Trainable params: 14,859,376
Non-trainable params: 0
____________________________________________

In [ ]:
from keras.callbacks import ModelCheckpoint, CSVLogger
#comparison_metric = MyAccuracy()
#checkpoint_filepath = "/content/drive/MyDrive/Weights/weights-improvement-{epoch:02d}-{val_my_accuracy:.2f}.hdf5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath = "/content/drive/MyDrive/MnistResults/vgg_best_weightsB.hdf5",
        save_weights_only=True,
        monitor="val_accuracy",
        #monitor="val_my_accuracy_19",
        mode='max',
        save_best_only=True)
model_checkpoint_callback2 = tf.keras.callbacks.ModelCheckpoint(
        filepath = "/content/drive/MyDrive/MnistResults/cnn_latest_weights4.hdf5",
        save_weights_only=True,
        monitor="val_accuracy",
        mode='max',
        save_best_only=False)
log_csv = CSVLogger("/content/drive/MyDrive/MnistResults/cnn_mylogs4.csv", separator = ",", append = False)
callback_list = [model_checkpoint_callback, model_checkpoint_callback2, log_csv]

In [ ]:
import time
class TimeHistory(K.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)
time_callback = TimeHistory()
history = model2.fit( train_X,train_labels, epochs= 10,validation_split=0.1, callbacks = [time_callback])
times = time_callback.times
print(times)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/keras/backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


1688/1688 [==============================] - 65s 28ms/step - loss: 0.7341 - accuracy: 0.7338 - val_loss: 0.1280 - val_accuracy: 0.9728
Epoch 2/10
1688/1688 [==============================] - 48s 28ms/step - loss: 0.1629 - accuracy: 0.9624 - val_loss: 0.0772 - val_accuracy: 0.9815
Epoch 3/10
1688/1688 [==============================] - 48s 28ms/step - loss: 0.1307 - accuracy: 0.9711 - val_loss: 0.0643 - val_accuracy: 0.9842
Epoch 4/10
1688/1688 [==============================] - 49s 29ms/step - loss: 0.1000 - accuracy: 0.9786 - val_loss: 0.0525 - val_accuracy: 0.9880
Epoch 5/10
1688/1688 [==============================] - 48s 29ms/step - loss: 0.0828 - accuracy: 0.9824 - val_loss: 0.0742 - val_accuracy: 0.9843
Epoch 6/10
1688/1688 [==============================] - 48s 29ms/step - loss: 0.1040 - accuracy: 0.9791 - val_loss: 0.0500 - val_accuracy: 0.9887
Epoch 7/10
1688/1688 [==============================] - 48s 29ms/step - loss: 0.0717 - accuracy: 0.9852 - val_loss: 0.0726 - val_accura

In [ ]:
average_vgg = sum(times)/len(times)
print(average_vgg)

50.03862862586975


In [ ]:
print("training time for vgg: " + str((average_vgg/54000)*1000) + " ms")

training time for vgg: 0.9266412708494397ms


In [ ]:
print(average_vgg/(54000/32)*1000)

29.65252066718207


In [ ]:
model2.load_weights("/content/drive/MyDrive/MnistResults/vgg_best_weightsB.hdf5")

In [ ]:
h = model2.evaluate(test_X, test_labels, verbose = 1)

/usr/local/lib/python3.7/dist-packages/keras/backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


313/313 [==============================] - 17s 8ms/step - loss: 0.0408 - accuracy: 0.9903


In [ ]:
print("evaluation time for vgg: " + str((3/test_X.shape[0])*1000) + "ms")

evaluation time for vgg: 0.3ms


## evaluation time for vgg: 0.3ms

In [ ]:
model2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model2.load_weights("/content/drive/MyDrive/MnistResults/vgg_best_weightsB.hdf5")

In [ ]:
# y_pred = model2.predict(test_X)
# #print(y_pred)

# import sklearn
# from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
# #print(confusion_matrix(test_labels, y_pred))
# print(f"accuracy = {accuracy_score(test_labels, y_pred)}")

In [ ]:
vgg_keras_file = "/content/drive/MyDrive/MnistResults/vgg_basemodel32X32_1.h5"
tf.keras.models.save_model(model2, vgg_keras_file, include_optimizer=False)
print('Saved baseline model to:', vgg_keras_file)

Saved baseline model to: /content/drive/MyDrive/MnistResults/vgg_basemodel32X32_1.h5


## size of VGG .h5 base model = 57M

In [ ]:
vgg_keras_file_tf = "/content/drive/MyDrive/MnistResults/vgg_basemodel32X32_1.tf"
tf.keras.models.save_model(model2, vgg_keras_file_tf, include_optimizer=False)
print('Saved baseline model to:', vgg_keras_file_tf)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/MnistResults/vgg_basemodel32X32_1.tf/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/MnistResults/vgg_basemodel32X32_1.tf/assets


Saved baseline model to: /content/drive/MyDrive/MnistResults/vgg_basemodel32X32_1.tf


# VGG .tf base model size = 58M

In [ ]:
model2.load_weights("/content/drive/MyDrive/MnistResults/vgg_best_weightsB.hdf5")

In [ ]:
vgg_keras_file_tf = "/content/drive/MyDrive/MnistResults/vgg_basemodel32X32_withWeights.tf"
tf.keras.models.save_model(model2, vgg_keras_file_tf, include_optimizer=False)
print('Saved baseline model to:', vgg_keras_file_tf)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/MnistResults/vgg_basemodel32X32_withWeights.tf/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/MnistResults/vgg_basemodel32X32_withWeights.tf/assets


Saved baseline model to: /content/drive/MyDrive/MnistResults/vgg_basemodel32X32_withWeights.tf


In [ ]:
modelVgg = model2

# compression without pruning

In [ ]:
# Create the .tflite file
tflite_model_file = "/content/drive/MyDrive/MnistResults/vgg_compressed32X32_1.tflite"
converter = tf.lite.TFLiteConverter.from_keras_model(modelVgg)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpjs6a2a91/assets


# VGG .tflite Compressed without Pruning  = 15M

# Pruning VGG
```



In [ ]:
import tempfile
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

finalSparsity = 0.05
list_accuracy0 = []
while finalSparsity <= 0.95:
  model2.load_weights('/content/drive/MyDrive/MnistResults/vggtrain_best_weights2.hdf5')
  pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.0,
                                                                final_sparsity=finalSparsity,
                                                                begin_step=0,
                                                                end_step=end_step)
  }
  
  # Define model for pruning.
  model_for_pruning = prune_low_magnitude(model2, **pruning_params)

  # `prune_low_magnitude` requires a recompile.
  model_for_pruning.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  #model_for_pruning.summary()
  logdir = tempfile.mkdtemp()

  callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
  ]

  model_for_pruning.fit(train_X, train_labels,
                    batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                    callbacks=callbacks)
  _, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_X, test_labels, verbose=0)
  print(f'Pruned test accuracy at final_sparsity: at {finalSparsity} is {model_for_pruning_accuracy}')
  list_accuracy0.append(model_for_pruning_accuracy)
  finalSparsity += 0.05
print(list_accuracy0)

/usr/local/lib/python3.7/dist-packages/keras/engine/base_layer.py:2223: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Epoch 1/2


/usr/local/lib/python3.7/dist-packages/keras/backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


422/422 [==============================] - 31s 51ms/step - loss: 0.3138 - accuracy: 0.9014 - val_loss: 0.0511 - val_accuracy: 0.9903
Epoch 2/2
422/422 [==============================] - 20s 47ms/step - loss: 0.0311 - accuracy: 0.9931 - val_loss: 0.0464 - val_accuracy: 0.9912
Pruned test accuracy at final_sparsity: at 0.05 is 0.9915000200271606
Epoch 1/2
422/422 [==============================] - 28s 50ms/step - loss: 0.3316 - accuracy: 0.8883 - val_loss: 0.0475 - val_accuracy: 0.9917
Epoch 2/2
422/422 [==============================] - 20s 47ms/step - loss: 0.0292 - accuracy: 0.9939 - val_loss: 0.0528 - val_accuracy: 0.9917
Pruned test accuracy at final_sparsity: at 0.1 is 0.9911999702453613
Epoch 1/2
422/422 [==============================] - 29s 51ms/step - loss: 0.3252 - accuracy: 0.9034 - val_loss: 0.0714 - val_accuracy: 0.9862
Epoch 2/2
422/422 [==============================] - 20s 47ms/step - loss: 0.0375 - accuracy: 0.9923 - val_loss: 0.0470 - val_accuracy: 0.9927
Pruned test a

In [ ]:
import tempfile
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

finalSparsity = 0.05
list_accuracy0 = []
while finalSparsity <= 0.95:
  model2.load_weights('/content/drive/MyDrive/MnistResults/vggtrain_best_weights2.hdf5')
  pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.0,
                                                                final_sparsity=finalSparsity,
                                                                begin_step=0,
                                                                end_step=end_step)
  }
  
  # Define model for pruning.
  model_for_pruning = prune_low_magnitude(model2, **pruning_params)

  # `prune_low_magnitude` requires a recompile.
  model_for_pruning.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  #model_for_pruning.summary()
  logdir = tempfile.mkdtemp()

  callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
  ]

  model_for_pruning.fit(train_X, train_labels,
                    batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                    callbacks=callbacks)
  _, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_X, test_labels, verbose=0)
  print(f'Pruned test accuracy at final_sparsity: at {finalSparsity} is {model_for_pruning_accuracy}')
  list_accuracy0.append(model_for_pruning_accuracy)
  finalSparsity += 0.05
print(list_accuracy0)

/usr/local/lib/python3.7/dist-packages/keras/engine/base_layer.py:2223: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Epoch 1/2


/usr/local/lib/python3.7/dist-packages/keras/backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


422/422 [==============================] - 28s 50ms/step - loss: 0.3284 - accuracy: 0.8901 - val_loss: 0.1037 - val_accuracy: 0.9843
Epoch 2/2
422/422 [==============================] - 20s 47ms/step - loss: 0.0569 - accuracy: 0.9893 - val_loss: 0.0549 - val_accuracy: 0.9923
Pruned test accuracy at final_sparsity: at 0.05 is 0.9901999831199646
Epoch 1/2
422/422 [==============================] - 28s 50ms/step - loss: 0.2917 - accuracy: 0.9062 - val_loss: 0.1172 - val_accuracy: 0.9913
Epoch 2/2
422/422 [==============================] - 20s 47ms/step - loss: 0.0508 - accuracy: 0.9896 - val_loss: 0.0419 - val_accuracy: 0.9913
Pruned test accuracy at final_sparsity: at 0.1 is 0.9904999732971191
Epoch 1/2
422/422 [==============================] - 29s 51ms/step - loss: 0.3092 - accuracy: 0.9064 - val_loss: 0.0551 - val_accuracy: 0.9898
Epoch 2/2
422/422 [==============================] - 20s 47ms/step - loss: 0.0319 - accuracy: 0.9934 - val_loss: 0.0498 - val_accuracy: 0.9927
Pruned test a

In [ ]:
print(list_accuracy0)

[0.9901999831199646, 0.9904999732971191, 0.992900013923645, 0.9918000102043152, 0.9923999905586243, 0.9921000003814697, 0.9878000020980835, 0.9884999990463257, 0.9921000003814697, 0.9911999702453613, 0.9925000071525574, 0.8859000205993652, 0.9871000051498413, 0.9865999817848206, 0.8871999979019165, 0.9840999841690063, 0.9883999824523926, 0.8840000033378601]


In [ ]:
print(list_accuracy0)

[0.9915000200271606, 0.9911999702453613, 0.9925000071525574, 0.9921000003814697, 0.9837999939918518, 0.9908999800682068, 0.988099992275238, 0.9901999831199646, 0.9909999966621399, 0.9907000064849854, 0.9907000064849854, 0.9853000044822693, 0.8016999959945679, 0.9854000210762024, 0.9858999848365784, 0.8931999802589417, 0.9901999831199646, 0.8820000290870667]


In [ ]:
import tensorflow_model_optimization as tfmot
import tempfile
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs



model2.load_weights('/content/drive/MyDrive/MnistResults/vgg_best_weightsB.hdf5')
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.20,
                                                              final_sparsity= 0.65 ,
                                                              begin_step=0,
                                                              end_step=end_step)
}

# Define model for pruning.
model_for_pruning = prune_low_magnitude(model2, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

#model_for_pruning.summary()
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_X, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
  test_X, test_labels, verbose=1)


/usr/local/lib/python3.7/dist-packages/keras/engine/base_layer.py:2223: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Epoch 1/2


/usr/local/lib/python3.7/dist-packages/keras/backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


  6/422 [..............................] - ETA: 1:37 - loss: 0.0456 - accuracy: 0.9909WARNING:tensorflow:Callback method `on_train_batch_begin` is slow compared to the batch time (batch time: 0.0352s vs `on_train_batch_begin` time: 0.0615s). Check your callbacks.


422/422 [==============================] - 37s 68ms/step - loss: 0.0420 - accuracy: 0.9916 - val_loss: 0.0418 - val_accuracy: 0.9920
Epoch 2/2
313/313 [==============================] - 3s 10ms/step - loss: 0.0329 - accuracy: 0.9932


In [ ]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
  test_X, test_labels, verbose=1)


313/313 [==============================] - 3s 10ms/step - loss: 0.0329 - accuracy: 0.9932


# VGG pruned model accuracy = 99.1%

In [ ]:
print("evaluation time for prunned vgg model: " + str((3/test_X.shape[0])*1000) + "ms")

evaluation time for prunned vgg model: 0.3ms


# evaluation time for prunned vgg model: 0.3ms, same as base model

In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
model_for_export.summary()
pruned_keras_file = "/content/drive/MyDrive/MnistResults/vgg_pruned32X32_1.tf"
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dense_1 (Dense)              (None, 50)                12850     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                510       
Total params: 14,859,376
Trainable params: 14,859,376
Non-trainable params: 0
____________________________________________

INFO:tensorflow:Assets written to: /content/drive/MyDrive/MnistResults/vgg_pruned32X32_1.tf/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/MnistResults/vgg_pruned32X32_1.tf/assets


Saved pruned Keras model to: /content/drive/MyDrive/MnistResults/vgg_pruned32X32_1.tf


## VGG Pruned 58M. same as original size

# now Compression after pruning

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model_vgg = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model_vgg)

print('Saved pruned TFLite model to:', pruned_tflite_file)


INFO:tensorflow:Assets written to: /tmp/tmpcxsi305e/assets


INFO:tensorflow:Assets written to: /tmp/tmpcxsi305e/assets


Saved pruned TFLite model to: /tmp/tmp1kjbqoik.tflite


In [ ]:
!du -sh /content/drive/MyDrive/MnistResults/*

27M	/content/drive/MyDrive/MnistResults/best_weights4.hdf5
27M	/content/drive/MyDrive/MnistResults/best_weights5.hdf5
92M	/content/drive/MyDrive/MnistResults/caps32X32_basemodel_1.tf
31M	/content/drive/MyDrive/MnistResults/caps32X32_basemodel_2.tf
31M	/content/drive/MyDrive/MnistResults/caps32X32_best_weights5.hdf5
16M	/content/drive/MyDrive/MnistResults/caps32X32_compressedB.tflite
4.0K	/content/drive/MyDrive/MnistResults/caps32X32mylogs5.csv
16M	/content/drive/MyDrive/MnistResults/caps32X32_P&Q_1_noZip.tflite
16M	/content/drive/MyDrive/MnistResults/caps32X32_P&Q_1.tflite
31M	/content/drive/MyDrive/MnistResults/caps32X32pruned_fileB.h5
31M	/content/drive/MyDrive/MnistResults/caps32X32pruned_fileB.tf
3.0K	/content/drive/MyDrive/MnistResults/caps48X48mylogs5.csv
27M	/content/drive/MyDrive/MnistResults/caps_basemodel28X28_1.h5
1.0K	/content/drive/MyDrive/MnistResults/caps_basemodelA.h5
27M	/content/drive/MyDrive/MnistResults/caps_basemodelA.tf
27M	/content/drive/MyDrive/MnistResults/caps

In [ ]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

#print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

INFO:tensorflow:Assets written to: /tmp/tmpr524givv/assets


INFO:tensorflow:Assets written to: /tmp/tmpr524givv/assets


Saved quantized and pruned TFLite model to: /tmp/tmpr9lefqp3.tflite
Size of gzipped pruned and quantized TFlite model: 7155959.00 bytes


In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size("/content/drive/MyDrive/MnistResults/vgg_basemodel32X32_withWeights.tf")))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size("/content/drive/MyDrive/MnistResults/vgg_pruned32X32_1.tf")))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(tflite_model_file)))

Size of gzipped baseline Keras model: 236.00 bytes
Size of gzipped pruned Keras model: 210.00 bytes
Size of gzipped pruned TFlite model: 11521622.00 bytes


# Quantized, Pruned and Zipped = 11.5M

In [ ]:
tflite_model_file = "/content/drive/MyDrive/MnistResults/vgg32X32_P&Q_1_noZip.tflite"
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp8wnrcfe2/assets


INFO:tensorflow:Assets written to: /tmp/tmp8wnrcfe2/assets


# Quantized and Pruned tflite model VGG 32X32 = 15M

In [ ]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on ever y image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_X[:1000]):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels[:1000]).mean()
  return accuracy

In [ ]:
interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Pruned and quantized TFLite test_accuracy:', test_accuracy)
print('Pruned TF test accuracy:', model_for_pruning_accuracy)

Evaluated on 0 results so far.


Pruned and quantized TFLite test_accuracy: 0.993
Pruned TF test accuracy: 0.9932000041007996


In [ ]:
!du -sh /content/drive/MyDrive/MnistResults/*

# =================================================================================================

# Capsule Networn 32X32

In [ ]:
caps1_n_maps = 32
caps1_n_caps = caps1_n_maps * 8 * 8  # 1152 primary capsules
caps1_n_dims = 8
caps2_n_caps = 10
caps2_n_dims = 16

tf.random.set_seed(500000)

In [ ]:
#class SquashLayer(K.layers.Layer, tfmot.sparsity.keras.PrunableLayer):
class SquashLayer(K.layers.Layer):
  def __init__(self, axis=-1, **kwargs):
    super(SquashLayer, self).__init__(**kwargs)
    self.axis = axis
    
  def build(self, input_shapes):
    pass

  """
  def get_prunable_weights(self):
    return []
  """

  def call(self, inputs):
    EPSILON = 1.0e-9
    squared_norm = tf.compat.v1.reduce_sum(tf.square(inputs),\
                                           axis=self.axis,\
                                           keepdims=True)
    safe_norm = tf.sqrt(squared_norm + EPSILON)
    squash_factor = squared_norm / (1. + squared_norm)
    unit_vector = inputs / safe_norm
    return squash_factor * unit_vector

  def get_config(self):
    config = super(SquashLayer, self).get_config()
    config.update({"axis": self.axis})
    return config

 
#class SafeNorm(K.layers.Layer, tfmot.sparsity.keras.PrunableLayer):
class SafeNorm(K.layers.Layer):
  
  def __init__(self, axis=-1, keep_dims = False,  **kwargs):
    super(SafeNorm, self).__init__(**kwargs)
    self.axis = axis
    self.keep_dims = keep_dims

  def build(self, input_shapes):
    pass

  """
  def get_prunable_weights(self):
    return []
  """

  def call(self, input):
    EPSILON = 1.0e-9
    squared_norm = tf.compat.v1.reduce_sum(tf.square(inputs),\
                                           axis=self.axis,\
                                           keepdims= self.keep_dims)
    safe_norm = tf.sqrt(squared_norm + EPSILON)
    return safe_norm

  def get_config(self):
    config = super(SafeNorm, self).get_config()
    config.update({"axis": self.axis, "keep_dims": self.keep_dims})
    return config
  
# This should be the part where the digit layer, and where we tile things
# This is incomplete, and work in progress
# TODO: Complete this
class MyDigitCapsLayer(K.layers.Layer, tfmot.sparsity.keras.PrunableLayer):
  def __init__(self, **kwargs):
    super(MyDigitCapsLayer, self).__init__(**kwargs)
  
  def get_config(self):
    config =  super(MyDigitCapsLayer, self).get_config()
    return config
  

  def build(self, input_shapes):
    init_sigma = 0.1  # TODO: use
    self.kernel = self.add_weight(\
                      "kernel",\
                      (caps1_n_caps, caps2_n_caps, caps2_n_dims, caps1_n_dims),\
                      initializer="random_normal",\
                      dtype=tf.float32)

  
  # To debug this function, I used prints to print the shape
  # expand_dims just adds an exis, so if you say expand_dims(inshape=(5, 3), -1),
  # you get the output shape (5, 3, 1), it just adds an axis at the end
  # Then tile just multiplies one of the dimensions (that is it stacks along that direction N times)
  # so tile(inshape=(5, 3, 1), [1, 1, 1000]) will yield a shape (5, 3, 1000)
  #
  # Notice I didn't tile in build, but in call, Most probaly this is the right thing to do
  # but we'll only figure out when we actually train
  def get_prunable_weights(self):
    return [self.kernel]
  
  def call(self, inputs):
    # Add a dimension at the end
    exp1 = tf.expand_dims(inputs, -1, name="caps1_output_expanded")
    # add a dimension along 3rd axis
    exp1 = tf.expand_dims(exp1, 2, name="caps2_output_espanced")
    # tile along 3rd axis
    tile = tf.tile(exp1, [1, 1, caps2_n_caps, 1, 1], name="caps1_output_tiled")
    caps2_predicted = tf.matmul(self.kernel, tile, name="caps2_predicted")
    return caps2_predicted

# https://www.tensorflow.org/api_docs/python/tf/keras/losses/Loss
class MarginLoss(K.losses.Loss):
    def __init__(self, **kwargs):
      super(MarginLoss, self).__init__(**kwargs)

  
    def get_config(self):
      config = super(MarginLoss, self).get_config()
      return config
    
    def safe_norm(self, input, axis=-2, epsilon=1e-5, keep_dims=False, name=None):
      squared_norm = tf.reduce_sum(tf.square(input), axis=axis,
                                     keepdims=keep_dims)
      return tf.sqrt(squared_norm + epsilon)

    """
    def get_prunable_weights(self):
      return []
    """

    def call(self,y_true, input):
      # print(f"y_true.shape = {y_true.shape}, y_pred.shape = {y_pred.shape}")
      # return K.losses.MeanSquaredError()(y_true, y_pred)

      #y_true = K.Input(shape=[], dtype=tf.int64, name="y")
      m_plus = 0.9
      m_minus = 0.1
      lambda_ = 0.5 
      
      #y_true one hot encode y_train
      T = tf.one_hot(y_true, depth=caps2_n_caps, name="T")
      
      caps2_output_norm = self.safe_norm(input, keep_dims = True)

      present_error_raw = tf.square(\
                                    tf.maximum(0., m_plus - caps2_output_norm),
                                    name="present_error_raw")
      present_error = tf.reshape(\
                                    present_error_raw, shape=(-1, 10),
                                    name="present_error")  
  
      absent_error_raw = tf.square(\
                                    tf.maximum(0., caps2_output_norm - m_minus),
                                    name="absent_error_raw")
      absent_error = tf.reshape(\
                                    absent_error_raw, shape=(-1, 10),
                                    name="absent_error")
  
      L = tf.add(\
                  T * present_error,\
                  lambda_ * (1.0 - T) * absent_error,
                  name="L")
      marginLoss = tf.reduce_mean(\
                                  tf.reduce_sum(L, axis=1),\
                                  name="margin_loss")
      return marginLoss


#class RoutingByAgreement(K.layers.Layer, tfmot.sparsity.keras.PrunableLayer):
class RoutingByAgreement(K.layers.Layer):
  def __init__(self, round_number=-1, **kwargs):
    super(RoutingByAgreement, self).__init__(**kwargs)
    self.round_number = round_number 

  def get_config(self):
    config = super(RoutingByAgreement, self).get_config()
    config.update({"round_number": self.round_number})
    return config
 

  def build(self, input_shapes):
    self.raw_weights_1 = self.add_weight("raw_weights", \
                                         (caps1_n_caps, caps2_n_caps, 1, 1), \
                                         initializer = "zeros", \
                                         dtype=tf.float32,)
    
    #print("Routing layer: self.raw_weights = ", self.raw_weights.shape, "input_shape = ", input_shapes)

  
  def get_prunable_weights(self):
    return [self.raw_weights_1]


  @staticmethod
  def squash(inputs):
    EPSILON = 1.0e-5
    squared_norm = tf.compat.v1.reduce_sum(tf.square(inputs),\
                                           keepdims=True)
    safe_norm = tf.sqrt(squared_norm + EPSILON)
    squash_factor = squared_norm / (1. + squared_norm)
    unit_vector = inputs / safe_norm
    return squash_factor * unit_vector

  def single_round_routing(self, inputs, raw_weights, agreement):
    raw_weights = tf.add(raw_weights, agreement)
    routing_wt = tf.nn.softmax(raw_weights, axis=2)
    wt_predictions = tf.multiply(routing_wt, inputs)
    wt_sum = tf.reduce_sum(wt_predictions, axis=1, keepdims=True)
    return wt_sum

  def call(self, inputs):
    agreement = tf.zeros(shape=self.raw_weights_1.shape)
    sqsh_wt_sum = None
    x = inputs
    for i in range(2):
      wt_sum = self.single_round_routing(inputs, self.raw_weights_1, agreement)
      sqsh_wt_sum = RoutingByAgreement.squash(wt_sum)
      sqsh_wt_sum_tiled = tf.tile(\
                          sqsh_wt_sum ,\
                          [1, caps1_n_caps, 1, 1, 1],\
                          name="caps2_output_round_1_tiled")
      agreement = tf.matmul(\
                            x, \
                            sqsh_wt_sum_tiled,\
                            transpose_a=True,\
                            name="agreement")
    return sqsh_wt_sum

class MyAccuracy(K.metrics.Metric):
  def __init__(self, **kwargs):
    super(MyAccuracy, self).__init__(**kwargs)
    self.acc_obj = K.metrics.Accuracy()
    self.state = 0
  

  def get_config(self):
    config = super(MyAccuracy, self).get_config()
    config.update({"acc_obj": None, "state": self.state})
    return config
  

  def safe_norm(self, input, axis=-2, epsilon=1e-5, keep_dims=True, name=None):
      squared_norm = tf.reduce_sum(tf.square(input), axis=axis,
                                     keepdims=keep_dims)
      return tf.sqrt(squared_norm + epsilon)

  def update_state(self, y_true, input, sample_weight=None):
    if self.acc_obj is None:
      self.acc_obj = K.metrics.Accuracy()
    y_proba = self.safe_norm(input, axis=-2)
    y_proba_argmax = tf.argmax(y_proba, axis=2)
    y_pred = tf.squeeze(y_proba_argmax, axis=[1,2], name="y_pred")
    #y_true = tf.reshape(y_true, (y_true.shape[0], ))
    y_true = tf.cast(y_true, dtype=tf.int64)
    self.acc_obj.update_state(y_true, y_pred, sample_weight)

  def reset_state(self):
    self.acc_obj.reset_state()

  def result(self):
    return self.acc_obj.result()

class MyReshapeLayer(K.layers.Layer):
  def __init__(self, axis=-1, keep_dims = False,  **kwargs):
    super(MyReshapeLayer, self).__init__(**kwargs)

  def build(self, input_shapes):
    pass

  def safe_norm(self, input, axis=-2, epsilon=1e-5, keep_dims=True, name=None):
      squared_norm = tf.reduce_sum(tf.square(input), axis=axis,
                                     keepdims=keep_dims)
      return tf.sqrt(squared_norm + epsilon)

  def call(self, input):
    print('printing shapes ------------------- ')
    EPSILON = 1.0e-9
    print(input)
    y_proba = self.safe_norm(input, axis=-2)
    print(y_proba)
    y_proba_argmax = tf.argmax(y_proba, axis=2)
    print(y_proba_argmax)
    y_pred = tf.squeeze(y_proba_argmax, axis=[1,2], name="y_pred")
    print(y_pred)
    return tf.cast(y_pred, tf.int64)

  def get_config(self):
    config = super(MyReshapeLayer, self).get_config()
    return config
  

In [ ]:
import keras
tf.random.set_seed(500000)
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = K.datasets.mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0
train_X =[]

for i in range(train_images.shape[0]):
  train_pad = np.pad(train_images[i], pad_width=2 , mode= 'edge')
  #print(train_pad)
  train_X.append(train_pad)
#PADDING test images
test_X = []
for i in range(test_images.shape[0]):
  test_pad = np.pad(test_images[i], pad_width=2 , mode= 'edge')
  #print(train_pad)
  test_X.append(test_pad)
train_X = np.array(train_X)
test_X = np.array(test_X)

In [ ]:
x_train = train_X
x_test = test_X
x_train.shape, x_test.shape

((60000, 32, 32), (10000, 32, 32))

In [ ]:
class Model:
    @staticmethod
    def build(inshape=(32, 32, 1)):
        inp = K.Input(shape=inshape, dtype=tf.float32, name='input')
        
        # Primary capsules
        # For each digit in the batch
        # 32 maps, each 6x6 grid of 8 dimensional vectors
        
        # First Conv layer
        conv1_params = \
        {
            "filters": 256,
            "kernel_size": 9,
            "strides": 1,
            "padding": "valid",
            "activation": tf.nn.relu,
        }
        x = K.layers.Conv2D(**conv1_params, name="conv_layer_1")(inp)
        
        # Second conv layer
        conv2_params = \
        {
            "filters": caps1_n_maps * caps1_n_dims, # 256 convolutional filters
            "kernel_size": 9,
            "strides": 2,
            "padding": "valid",
            "activation": tf.nn.relu
        }
        x = K.layers.Conv2D(**conv2_params, name="conv_layer_2")(x)
        
        # Reshape
        x = K.layers.Reshape(\
                             (caps1_n_caps, caps1_n_dims),\
                             name="reshape_layer_1")(x)
                             
        x = SquashLayer(name="caps1_output_layer")(x)
        
        x = MyDigitCapsLayer(name = "caps2_predicted")(x)
        caps2_predicted = x # Save this value for later
        
        #routing by agreement (2 rounds)
        x = RoutingByAgreement(name="routing1", round_number=2)(x)
        
        return K.Model(inputs=inp, outputs=x, name='my')
    
m = Model.build()
print(m.summary())


Model: "my"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 32, 32, 1)]       0         
_________________________________________________________________
conv_layer_1 (Conv2D)        (None, 24, 24, 256)       20992     
_________________________________________________________________
conv_layer_2 (Conv2D)        (None, 8, 8, 256)         5308672   
_________________________________________________________________
reshape_layer_1 (Reshape)    (None, 2048, 8)           0         
_________________________________________________________________
caps1_output_layer (SquashLa (None, 2048, 8)           0         
_________________________________________________________________
caps2_predicted (MyDigitCaps (None, 2048, 10, 16, 1)   2621440   
_________________________________________________________________
routing1 (RoutingByAgreement (None, 1, 10, 16, 1)      20480    

In [ ]:
m.compile(optimizer='adam', loss=MarginLoss(), metrics=[MyAccuracy()])

In [ ]:
import time
class TimeHistory(K.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)
time_callback = TimeHistory()
history = m.fit(x_train, train_labels, batch_size=32, epochs=10, verbose= 1, validation_split=0.1, callbacks = [time_callback])
times = time_callback.times
print(times)

Epoch 1/10
1688/1688 [==============================] - 224s 132ms/step - loss: 2.2778 - my_accuracy: 0.9248 - val_loss: 2.2755 - val_my_accuracy: 0.9653
Epoch 2/10
1688/1688 [==============================] - 219s 130ms/step - loss: 2.2778 - my_accuracy: 0.9370 - val_loss: 2.2755 - val_my_accuracy: 0.9720
Epoch 3/10
1688/1688 [==============================] - 218s 129ms/step - loss: 2.2778 - my_accuracy: 0.9439 - val_loss: 2.2756 - val_my_accuracy: 0.9772
Epoch 4/10
1688/1688 [==============================] - 217s 129ms/step - loss: 2.2778 - my_accuracy: 0.9293 - val_loss: 2.2755 - val_my_accuracy: 0.9593
Epoch 5/10
1688/1688 [==============================] - 217s 129ms/step - loss: 2.2778 - my_accuracy: 0.9461 - val_loss: 2.2754 - val_my_accuracy: 0.9807
Epoch 6/10
1688/1688 [==============================] - 217s 129ms/step - loss: 2.2778 - my_accuracy: 0.9455 - val_loss: 2.2757 - val_my_accuracy: 0.9310
Epoch 7/10
1688/1688 [==============================] - 217s 129ms/step - lo

In [ ]:
average = sum(times)/len(times)
print(average)
print("training time per instance for capsule network : " + str((average/54000)*1000)+ " ms")


218.1523354291916
training time per instance for capsule network : 4.039858063503548 ms


In [ ]:
import time
class TimeHistory(K.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)
time_callback = TimeHistory()
history = m.fit(x_train, train_labels, batch_size=32, epochs=2, verbose= 1, validation_split=0.1, callbacks = [time_callback])
times = time_callback.times
print(times)

Epoch 1/2
1688/1688 [==============================] - 233s 130ms/step - loss: 2.2778 - my_accuracy: 0.9252 - val_loss: 2.2755 - val_my_accuracy: 0.9652
Epoch 2/2
1688/1688 [==============================] - 218s 129ms/step - loss: 2.2778 - my_accuracy: 0.9369 - val_loss: 2.2755 - val_my_accuracy: 0.9702
[233.21886086463928, 218.15961813926697]


In [ ]:
m.load_weights("/content/drive/MyDrive/MnistResults/caps32X32_best_weights5.hdf5")


In [ ]:
m.save("/content/drive/MyDrive/MnistResults/caps32X32_basemodel_3.tf", save_format='tf')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/MnistResults/caps32X32_basemodel_3.tf/assets


In [ ]:
m.evaluate(x_test, test_labels, verbose = 1)

313/313 [==============================] - 19s 61ms/step - loss: 2.2765 - my_accuracy: 0.9745


[2.276534080505371, 0.9745000004768372]

In [ ]:
print("time for caps model evaluation: " + str((19/test_X.shape[0])*1000) + "ms")

time for caps model evaluation: 1.9ms


# pruning 65%

In [ ]:
import tensorflow_model_optimization as tfmot
import tempfile

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# TODO: Remove this line
#m.load_weights('/content/drive/MyDrive/MnistResults/best_weights4.hdf5')

# print("ORIGINAL MODEL")
# print(mm.summary())
# print('-' * 80)

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = x_train.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

mm = m
mm.load_weights('/content/drive/MyDrive/MnistResults/caps32X32_best_weights5.hdf5')

# Define model for pruning.
# pruning_params = {
#       'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
#                                                                final_sparsity=0.80,
#                                                                begin_step=0,
#                                                                end_step=end_step)
# }

pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.20,
                                                              final_sparsity=0.65,
                                                              begin_step=0,
                                                              end_step=end_step)
}

"""
model_for_pruning = prune_low_magnitude(mm, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=MarginLoss(), metrics=[MyAccuracy()])

model_for_pruning.summary()

"""


######################################################################
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

# Helper function uses `prune_low_magnitude` to make only the 
# Dense layers train with pruning.
def apply_pruning_to_layers(layer):
  #print("called")
  if isinstance(layer, MyDigitCapsLayer):
    print(f"Layer {layer} {layer.name} slated for pruning")
    return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
  elif layer.name == "conv_layer_2":
    print(f"Layer {layer} {layer.name} slated for pruning")
    return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
  elif layer.name == "routing1":
    print(f"Layer {layer} {layer.name} slated for pruning")
    return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
  elif layer.name == "conv_layer_1":
    print(f"Layer {layer} {layer.name} slated for pruning")
    return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)


  print(f"Layer {layer} {layer.name} unchanged")
  return layer

# Use `tf.keras.models.clone_model` to apply `apply_pruning_to_layers` 
# to the layers of the model.
model_for_pruning = tf.keras.models.clone_model(
    mm,
    clone_function=apply_pruning_to_layers,
)
#print(model_for_pruning.summary())

"""
model_for_pruning = K.models.Sequential(\
              [\
                model_for_pruning,\
                MyReshapeLayer(),\
              ]\
)
"""


model_for_pruning.compile(optimizer='adam', loss=MarginLoss(), metrics=[MyAccuracy()])
#model_for_pruning.compile(optimizer='adam', loss=MarginLoss())

model_for_pruning.fit(x_train, train_labels,
                  batch_size=32, epochs=2, validation_split=validation_split,
                  callbacks=callbacks, verbose = 1)
#_, model_for_pruning_accuracy = model_for_pruning.evaluate(x_test, y_test, verbose=1)

Layer <keras.layers.convolutional.Conv2D object at 0x7f2121555790> conv_layer_1 slated for pruning
Layer <keras.layers.convolutional.Conv2D object at 0x7f2121570690> conv_layer_2 slated for pruning
Layer <keras.layers.core.Reshape object at 0x7f2121122cd0> reshape_layer_1 unchanged
Layer <__main__.SquashLayer object at 0x7f212144fcd0> caps1_output_layer unchanged
Layer <__main__.MyDigitCapsLayer object at 0x7f2121093a50> caps2_predicted slated for pruning
Layer <__main__.RoutingByAgreement object at 0x7f2121011810> routing1 slated for pruning


/usr/local/lib/python3.7/dist-packages/keras/engine/base_layer.py:2223: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Epoch 1/2
1688/1688 [==============================] - 225s 132ms/step - loss: 2.2778 - my_accuracy_3: 0.9310 - val_loss: 2.2755 - val_my_accuracy_3: 0.9643
Epoch 2/2
1688/1688 [==============================] - 218s 129ms/step - loss: 2.2778 - my_accuracy_3: 0.9442 - val_loss: 2.2755 - val_my_accuracy_3: 0.9723


In [ ]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(x_test, test_labels, verbose=1)

313/313 [==============================] - 18s 58ms/step - loss: 2.2765 - my_accuracy_3: 0.9704


In [ ]:
print("evaluation time for prunned model : " +str((18/x_test.shape[0])*1000) + "ms")

evaluation time for prunned model : 1.8ms


In [ ]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(x_test, test_labels, verbose=1)

313/313 [==============================] - 18s 59ms/step - loss: 2.2765 - my_accuracy_3: 0.9704


In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
model_for_export.summary()
pruned_keras_file = "/content/drive/MyDrive/MnistResults/caps32X32pruned_fileC.tf"
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Model: "my"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 32, 32, 1)]       0         
_________________________________________________________________
conv_layer_1 (Conv2D)        (None, 24, 24, 256)       20992     
_________________________________________________________________
conv_layer_2 (Conv2D)        (None, 8, 8, 256)         5308672   
_________________________________________________________________
reshape_layer_1 (Reshape)    (None, 2048, 8)           0         
_________________________________________________________________
caps1_output_layer (SquashLa (None, 2048, 8)           0         
_________________________________________________________________
caps2_predicted (MyDigitCaps (None, 2048, 10, 16, 1)   2621440   
_________________________________________________________________
routing1 (RoutingByAgreement (None, 1, 10, 16, 1)      20480    

INFO:tensorflow:Assets written to: /content/drive/MyDrive/MnistResults/caps32X32pruned_fileC.tf/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/MnistResults/caps32X32pruned_fileC.tf/assets


Saved pruned Keras model to: /content/drive/MyDrive/MnistResults/caps32X32pruned_fileC.tf


In [ ]:
model_for_export.compile(optimizer='adam', loss=MarginLoss(), metrics=[MyAccuracy()])

In [ ]:
model_for_export.evaluate(x_test, test_labels, verbose=1)

313/313 [==============================] - 19s 58ms/step - loss: 2.2765 - my_accuracy_4: 0.9704


[2.2765438556671143, 0.9703999757766724]

In [ ]:
#trying original weights on model_for_export
model_check = model_for_export
model_check.load_weights("/content/drive/MyDrive/MnistResults/caps32X32_best_weights5.hdf5")

In [ ]:
model_check.evaluate(x_test, test_labels, verbose=1)

313/313 [==============================] - 18s 59ms/step - loss: 2.2765 - my_accuracy_4: 0.9745


[2.276534080505371, 0.9745000004768372]

# Now compression after pruning

In [ ]:
#converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
tflite_model_file = "/content/drive/MyDrive/MnistResults/caps32X32_P&Q_1.tflite"
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]

tflite_model = converter.convert()
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpm7ovq2hb/assets


INFO:tensorflow:Assets written to: /tmp/tmpm7ovq2hb/assets


In [ ]:
tflite_model_file = "/content/drive/MyDrive/MnistResults/caps32X32_P&Q_2_noZip.tflite"
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]

tflite_model = converter.convert()
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp8cuedoab/assets


INFO:tensorflow:Assets written to: /tmp/tmp8cuedoab/assets


In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size("/content/drive/MyDrive/MnistResults/caps32X32_basemodel_1.tf")))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(tflite_model_file)))

Size of gzipped baseline Keras model: 218.00 bytes
Size of gzipped pruned Keras model: 218.00 bytes
Size of gzipped pruned TFlite model: 14577587.00 bytes


In [ ]:
!du -sh /content/drive/MyDrive/MnistResults/*

27M	/content/drive/MyDrive/MnistResults/best_weights4.hdf5
27M	/content/drive/MyDrive/MnistResults/best_weights5.hdf5
92M	/content/drive/MyDrive/MnistResults/caps32X32_basemodel_1.tf
31M	/content/drive/MyDrive/MnistResults/caps32X32_basemodel_2.tf
92M	/content/drive/MyDrive/MnistResults/caps32X32_basemodel_3.tf
31M	/content/drive/MyDrive/MnistResults/caps32X32_best_weights5.hdf5
16M	/content/drive/MyDrive/MnistResults/caps32X32_compressedB.tflite
4.0K	/content/drive/MyDrive/MnistResults/caps32X32mylogs5.csv
16M	/content/drive/MyDrive/MnistResults/caps32X32_P&Q_1_noZip.tflite
16M	/content/drive/MyDrive/MnistResults/caps32X32_P&Q_1.tflite
16M	/content/drive/MyDrive/MnistResults/caps32X32_P&Q_2_noZip.tflite
31M	/content/drive/MyDrive/MnistResults/caps32X32pruned_fileB.h5
31M	/content/drive/MyDrive/MnistResults/caps32X32pruned_fileB.tf
31M	/content/drive/MyDrive/MnistResults/caps32X32pruned_fileC.tf
3.0K	/content/drive/MyDrive/MnistResults/caps48X48mylogs5.csv
27M	/content/drive/MyDrive/Mn

In [ ]:
m.evaluate(test_X, test_labels, verbose = 1)

313/313 [==============================] - 20s 61ms/step - loss: 2.2765 - my_accuracy: 0.9745


[2.276534080505371, 0.9745000004768372]

In [ ]:
print("time for caps model evaluation: " + str((20/test_X.shape[0])*1000) + "ms")

time for caps model evaluation: 2.0ms


# Caps 32X32 base model evaluation time 2ms

In [ ]:
!du -sh /content/drive/MyDrive/MnistResults/*

27M	/content/drive/MyDrive/MnistResults/best_weights4.hdf5
27M	/content/drive/MyDrive/MnistResults/best_weights5.hdf5
92M	/content/drive/MyDrive/MnistResults/caps32X32_basemodel_1.tf
31M	/content/drive/MyDrive/MnistResults/caps32X32_basemodel_2.tf
31M	/content/drive/MyDrive/MnistResults/caps32X32_best_weights5.hdf5
16M	/content/drive/MyDrive/MnistResults/caps32X32_compressedB.tflite
4.0K	/content/drive/MyDrive/MnistResults/caps32X32mylogs5.csv
16M	/content/drive/MyDrive/MnistResults/caps32X32_P&Q_1_noZip.tflite
16M	/content/drive/MyDrive/MnistResults/caps32X32_P&Q_1.tflite
31M	/content/drive/MyDrive/MnistResults/caps32X32pruned_fileB.h5
31M	/content/drive/MyDrive/MnistResults/caps32X32pruned_fileB.tf
3.0K	/content/drive/MyDrive/MnistResults/caps48X48mylogs5.csv
27M	/content/drive/MyDrive/MnistResults/caps_basemodel28X28_1.h5
1.0K	/content/drive/MyDrive/MnistResults/caps_basemodelA.h5
27M	/content/drive/MyDrive/MnistResults/caps_basemodelA.tf
27M	/content/drive/MyDrive/MnistResults/caps